<a href="https://colab.research.google.com/github/ImVrishank/NLP/blob/main/GPT_2(explaining_preliminary_concepts).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download the tiny shakespeare dataset.
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


--2025-02-27 07:15:25--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  4.74MB/s    in 0.2s    

2025-02-27 07:15:26 (4.74 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# reading the same dataset
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
len(text)

1115394

In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# we need a sorted list of all the unique characters in the datset

chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# creating mapping for the tokenization of charecters in the dataset

s_to_i = {ch:i for i,ch in enumerate(chars)}
i_to_s = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [s_to_i[c] for c in s]
decode = lambda l: ''.join([i_to_s[i] for i in l])

# Tokenization:
as we know, ML needs numbers to work with and not charecters. Tokenization is basically taking a small defined unit of the dataset and assigning it a number. We need a defined map for the char to the integer and vice versa.
___
## In this project:
we are building a charecter based model, so our tokens are charecters themselves. we have 65 unique charecters in this dataset (all of them present in the chars list)

### variables:

* s_to_i is a dictionary that maps the perticular charecter to an integer.

* i_to_s is a dictionary that maps the perticular charecter to a charecter.

* encode:
  calls a function that takes in a string and gives each char one by one to the dictionary to eventually convert the entire string into a list of numbers, the list of numbers is the return value of encode.

* decode:
  does the same thing as encode, but coverts a list of integers into a string. works in the same way

In [ ]:
# example of encode's and decode's working
print(encode("hello"))
print(decode(encode("hello")))

[46, 43, 50, 50, 53]
hello


In [ ]:
# now that we have the mapping ready, we need to apply the mapping to the entire dataset, we also use tensors from torch to increase efficiency

import torch
data = torch.tensor(encode(text), dtype = torch.long)


In [ ]:
# looking at the working
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
# train test data split

split_fraction = int(0.9*len(data))
train_data = data[:split_fraction]
val_data = data[split_fraction:]

# Sampling data
Whenever we hand data to a model, we do not hand it the entire dataset, we have it chunks of data at a time. If we were to hand it out as a single entity, it would end being very computationally inefficient.

* block_size: it is the maximum size of the chunk of data we are handing to the model
*

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

Here we can observe why it would have been extremely inefficient to compute over the whole dataset. If we computed over the whole dataset, our final char would have all the other words as features and a singular lable, so as we move along the dataset, it would get harder to train more data.

Also, youve got to remember that the numbers above all corrospond to a certian char.


In [ ]:
# explaining how we are taking features and lables in the training dataset

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


# Transformers
When we hand data to transformers, it is done through tensors. and tensors usually contain multiple batches as shown above. All the batches are computed parallely and independently in the GPU.


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # corrosponds to how many batches we want to run parallely in the GPU
block_size = 8 # max length for each batch (as seen above)

def get_batch(split):
  data = train_data if split == 'train' else val_data # depending on wheter we are working on train data or validation data
  # we are generating 4 different numbers which will be the index of the first element of the 4 batches we are going to hand to the GPU.
  # [len(data) - block_size] beacuse if we take only len(data) we will observe overflow of data and hence an indexing error.
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i : i + block_size] for i in ix]) # this is the tensor of the 4 batches' features
  y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix]) # this is the tensor of the 4 batches' labels
  return x, y




In [ ]:
# testing the above:

xb, yb = get_batch('train')
print("input: ")
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)

print('---')
print("visualizing the data: \n")
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")


input: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
visualizing the data: 

when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 

# Bigram model:
starting off with the most simple model, it just looks at the previous char and predicts the next char. as it is this simple, the outputs it gives is horrible.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(num_embeddings = vocab_size, embedding_dim = vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) B stands for batch, T for token and C for charecters


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # preserving the shape of logits
            logits = logits.view(B*T, C) # changing the shape of the logits as cross_entropy expects in this way. explained elaborately in notes.
            targets = targets.view(B*T) # again, cross_entropy expects it in this way. you could just pass (-1) as the arguement, will work the same, but just for clarity
            # .view() to the plain eye works in the same way as .reshape() works. but for tensors .view() is more reliable. Not really worth going too deep into.
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # generate output.
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens): # _ beacuse the loop variable is insignificant (covention)
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1, 1)
print(decode(m.generate(idx, dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [ ]:
# using the pytorch optimizer

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32 # increased from 4
epochs = 10000
for steps in range(epochs):

  # get a batch of data
  xb, yb = get_batch('train')

  # calculating loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.5727508068084717


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht 


 # matrix trimming
 We are going to look at 4 ways of trimming a matrix such that it becomes a lower triangualar matrix.

 This is done because we want to only have the present token look at previous token for context and not bother about the token coming later, as the ones that come later are in the future.

 itll make more sense after you finish the next section as well.

In [ ]:
# toy example
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# initailizing toy example
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

we here have 4 batches, each containing 8 tokens. lets say we are at the 5th token of the 2nd batch. this token only needs to know what the 4th, 3rd, 2nd and 1st token is in the 2nd batch. it does not need to know what the 6th, 7th and 8th tokens are, beacuse those are yet to be guessed. the 1st, 2nd, 3rd, 4th act as the feature vector for the 5th token of the second batch.

In [ ]:
# method 1: using nested for loops, very inefficient
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# method 2: using matrix multiply
""" the tril method gives a lower triangular matrix, maintaining the lower triangle of the given matrix and making the rest of them 0"""
xbow2 = torch.zeros((B,T,C))
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)




In [ ]:
# method 3: using softmax to get a normalized lower triangular matrix with the sum of all rows being equal to 1
xbow3 = torch.zeros((B,T,C))
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x



# self attention:
until now, we spoke about how the wei matrixx will have equal weightages. that is, if we look at a row the sum of all elements of that row will be 1. and 1 will be equally distributed amonng all the non zero values as seen above.


but, we dont need it to be such that all of them are equal. that just means the affinity of the next char is equal to all the previous features(that is, the previous tokens). But the next char should not have equal affinity to all the prev chars. it should have greater affinity towards some previous chars and lower affinity to others, all this while still adding upto 1 in each row.

so we could have 0.6, 0.2, 0.2 as the affinities of the 4th token of a certain batch. instead of 0.33,0.33,0.33 as the affinities. This would mean that the 4th token has a lot of affinity to the first token and would drastically change depending on the value of the first token, and not as much depending on the second and third tokens. this could be imagined as the parameters from linear regression (theta). where theta is the set of weights and biases each weight corrosponding to a certain feature column.


___

each token gives out 2 vectors. One vector is called the QUERY vector and the other is called the KEY vector.

QUERY vector: what am i looking for? (what is the token looking for?)

KEY vector: what do i contain? (what does the token contain?)

this is used to get the wei vector. every QUERY vector takes the dot product with every KEY vector, this gives us the wei vector.

In [ ]:
# self attention!

torch.manual_seed(1337)
B, T, C = 4, 8, 32 #(Batch, Time, Channel) we have a 4 x 8 array of tokens, and each token has 32 dimensional information
x = torch.randn(B, T, C)

"""nn.Linear basically does the linear combination. i.e) W^t . x  + b. we do not need biases here do we say bias=False"""

# implementing for a single head of self attention:
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias = False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x) # ()

""" we take 16 for a reason, remember when initializing channel size we told that each token has a 32 dimensional information? this 32 dimensional info is going to be the
2 vectors KEY and QUERY. hence each has half of 32"""

wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T) [B remains a contant. we basically matrix multiply (T, 16) @ (16, T) --> (T, T)]

"""so first we take the matrix multiply of all the queries and tokens, then we use the technique we learnt about earlier to make sure only the ones before the
token (the context are taken into consideration and not all of them)"""


'so first we take the matrix multiply of all the queries and tokens, then we use the technique we learnt about earlier to make sure only the ones before the \ntoken (the context are taken into consideration and not all of them)'

In [ ]:
wei[0]

tensor([[-1.7629, -1.3011,  0.5652,  2.1616, -1.0674,  1.9632,  1.0765, -0.4530],
        [-3.3334, -1.6556,  0.1040,  3.3782, -2.1825,  1.0415, -0.0557,  0.2927],
        [-1.0226, -1.2606,  0.0762, -0.3813, -0.9843, -1.4303,  0.0749, -0.9547],
        [ 0.7836, -0.8014, -0.3368, -0.8496, -0.5602, -1.1701, -1.2927, -1.0260],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,  0.8638,  0.3719,  0.9258],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,  1.4187,  1.2196],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,  0.8048],
        [-1.8044, -0.4126, -0.8306,  0.5899, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)

In [ ]:
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [ ]:
out[:,:,0]

tensor([[-0.1571,  0.6764,  0.4823,  0.1971,  0.2510,  0.3428,  0.1866,  0.1301],
        [-1.3254, -0.3834, -1.1100, -0.9719,  0.3359,  0.1698,  0.0465, -0.5390],
        [ 0.0689, -0.4638, -0.6404, -0.6719, -0.0143, -0.6386, -0.2237, -0.2482],
        [ 0.0972,  0.1931,  0.2911,  0.1755,  0.1735,  0.2468, -0.3978, -0.3593]],
       grad_fn=<SelectBackward0>)

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])